In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-15 20:53:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-03-15 20:53:46 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
#Read in Vine_Table from S3 bucket
from pyspark import SparkFiles
url ="https://cnewell415-bucket.s3.amazonaws.com/vine_table.csv"
spark.sparkContext.addFile(url)
vine_df = spark.read.csv(SparkFiles.get("vine_table.csv"), sep=",", header=True, inferSchema=True)


In [7]:
# Show the created Dataframe
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RED72VWWCOS7S|          1|            2|          8|   N|                Y|
| RZHWQ208LTEPV|          5|            0|          0|   N|                Y|
|R37LBC3XAVLYOO|          5|            4|          5|   N|                Y|
|R3L7XJMA0MVJWC|          5|            0|          0|   N|                Y|
|R2I2GHSI7T1UBN|          1|            5|          6|   N|                Y|
|R2GFFKHK4I6VMX|          5|            0|          0|   N|                Y|
|R1R0UDX2XAN1S4|          4|            0|          0|   N|                Y|
|R22C8FMBSTFRY8|          5|            2|          2|   N|                Y|
|R118NNIQ75XPGO|          3|            0|          0|   N|                Y|
|R30HYXHZQ49621|          2|            0|          0|   N|     

In [13]:
# Check out the schema of the data
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [18]:
#Filter the vine_df for all entries with 20 or more total votes
helpful_reviews = vine_df.filter("total_votes >= 20").select("review_id", "star_rating", "helpful_votes", "total_votes" , "vine", "verified_purchase")
helpful_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|     

In [20]:
#Filter the helpful_reviews where the "helpful_votes / "total_votes" >= .50
#These are reviewers with 20 or more votes and more often than not, they are helpful
helpful_reviews_v2 = helpful_reviews.filter("helpful_votes / total_votes >= .50").select("review_id", "star_rating", "helpful_votes", "total_votes" , "vine", "verified_purchase")
helpful_reviews_v2.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R22G14MRW8D4JT|          1|           30|         32|   N|                N|
|R299KYRKCKSUR5|          5|           29|         30|   N|     

In [23]:
#Filter the reviews down to just helpful reviews by Vine members (paid reviews)
paid_reviews = helpful_reviews.filter("vine == 'Y'").select("review_id", "star_rating", "helpful_votes", "total_votes" , "vine", "verified_purchase")
paid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28DXTC3JQ9IY1|          4|           24|         26|   Y|                N|
|R3AFZKLQXATHBU|          5|           44|         49|   Y|                N|
|R2RUUF2JPJPC0E|          4|           20|         22|   Y|                N|
| RFZ2WUH4248AB|          2|           26|         27|   Y|                N|
|R1Q4LVHIFOWYFR|          5|           23|         28|   Y|                N|
| R4YEPTQED3X1Q|          5|           19|         20|   Y|                N|
|R2Z7C8YCRSC9DP|          5|           22|         22|   Y|                N|
|R3J8OI5CB74P5K|          1|           22|         25|   Y|                N|
| RH39LMKN6AZDC|          5|           33|         40|   Y|                N|
| R8RD8K0ESJSRD|          5|           21|         21|   Y|     

In [24]:
#Filter the reviews down to helpful reviews by non-vine members (unpaid reviews)
unpaid_reviews = helpful_reviews.filter("vine == 'N'").select("review_id", "star_rating", "helpful_votes", "total_votes" , "vine", "verified_purchase")
unpaid_reviews.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2RKCSAG6GBA4A|          1|            8|         28|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|     

In [30]:
# Total Number of Paid Reviews
paid_count = paid_reviews.count()
paid_count

452

In [32]:
# Number of 5-Star Paid Reviews
five_star_paid_reviews = paid_reviews.filter("star_rating == 5").count()
five_star_paid_reviews

210

In [33]:
# Percentage of Paid Reviews that are 5-Star
percent_five_star_paid_reviews = five_star_paid_reviews / paid_count
percent_five_star_paid_reviews

0.4646017699115044

In [34]:
# Total Number of Unpaid Reviews
unpaid_count = unpaid_reviews.count()
unpaid_count

570181

In [35]:
# Number of 5-Star Unpaid Reviews
five_star_unpaid_reviews = unpaid_reviews.filter("star_rating == 5").count()
five_star_unpaid_reviews

291254

In [36]:
# Percentage of Unpaid Reviews that are 5-Star
percent_five_star_unpaid_reviews = five_star_unpaid_reviews / unpaid_count
percent_five_star_unpaid_reviews

0.5108097253328329